# More on Decorators

In [41]:
import time
def slow_function(a,b):
    print("Sleeping...")
    s = time.time()
    time.sleep(5)
    e = time.time()
    print(e-s)
    return a+b
slow_function(5,5)

Sleeping...
5.000113248825073


10

## Real-world examples

In [ ]:
# When to use decorators? When you want to add some bit of code to multiple functions.

### Print the return type

In [8]:
def print_return_type(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print("{}() retutned type {}".format(func.__name__, type(result)))
        return result
    return wrapper

@print_return_type
def foo(value):
    return value
print(foo(42))
print(foo([1, 2, 3]))
print(foo({"a":42}))

foo() retutned type <class 'int'>
42
foo() retutned type <class 'list'>
[1, 2, 3]
foo() retutned type <class 'dict'>
{'a': 42}


### Counter

In [20]:
def counter(func):
    def wrapper(*args, **kwargs):
        wrapper.count +=1
        return wrapper.count
    wrapper.count = 0
    return wrapper
@counter
def foo():
    print("calling foo()")
@counter
def boo():
    print("calling boo()")
foo()
foo()

print("foo() was called {} times.".format(boo.count))

foo() was called 2 times.


## Decorators and metadata 

In [26]:
def add_hello(func):
    def wrapper(*args, **kwargs):
        print("Hello")
        return func(*args, **kwargs)
    return wrapper

@add_hello
def print_sum(a,b):
    """Adds two numbers and prints the sum"""
    print(a+b)
print_sum(10, 20)
print(print_sum.__doc__)
print("\n")


def add_hello(func):
    def wrapper(*args, **kwargs):
        """Print 'hello' and then call the decorated function."""
        print("Hello")
        return func(*args, **kwargs)
    return wrapper

@add_hello
def print_sum(a,b):
    """Adds two numbers and prints the sum"""
    print(a+b)
print_sum(10, 20)
print(print_sum.__doc__)
print("\n")


from functools import wraps
def add_hello(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        """Print 'hello' and then call the decorated function."""
        print("Hello")
        return func(*args, **kwargs)
    return wrapper

@add_hello
def print_sum(a,b):
    """Adds two numbers and prints the sum"""
    print(a+b)
print_sum(10, 20)
print(print_sum.__doc__)

Hello
30
None


Hello
30
Print 'hello' and then call the decorated function.


Hello
30
Adds two numbers and prints the sum


### Measuring decorator overhead

In [50]:
def check_everything(func):
  @wraps(func)
  def wrapper(*args, **kwargs):
    time.sleep(1.5)
    result = func(*args, **kwargs)
    return result
  return wrapper

@check_everything
def duplicate(my_list):
    return my_list + my_list

t_start = time.time()
duplicated_list = duplicate(list(range(50)))
t_end = time.time()
decorated_time = t_end - t_start

t_start = time.time()
duplicated_list = duplicate.__wrapped__(list(range(50)))
t_end = time.time()
undecorated = t_end - t_start
print("Decorated time: {:.5f}s".format(decorated_time))
print("Undecorated time: {:.5f}s".format(undecorated))

Decorated time: 1.50106s
Undecorated time: 0.00000s


## Decorators that take arguments

### Run_n_times() 

In [58]:
def run_n_times(n):
  def decorator(func):
    def wrapper(*args, **kwargs):
      for i in range(n):
        func(*args, **kwargs)
    return wrapper
  return decorator

@run_n_times(10)
def print_sum(a, b):
    print (a+b)

print_sum(15,20)

run_5_times = run_n_times(5)
@run_5_times
def print_sum(a, b):
    print(a+b)
print_sum(4, 100)


35
35
35
35
35
35
35
35
35
35
104
104
104
104
104


### HTML Generator

In [71]:
def html(open_tag, close_tag):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            msg = func(*args, **kwargs)
            return "{}{}{}".format(open_tag, msg, close_tag)
        return wrapper
    return decorator

@html("<b>", "</b>")
def hello(name):
    return "Hello {}!".format(name)
print(hello("Alice"))
print("\n")

@html("<i>", "</i>")
def goodbye(name):
    return "Goodbye {}.".format(name)
print(goodbye("Alice"))
print("\n")

@html("<div>","</div>")
def hello_goodbye(name):
    return "\n{}\n{}\n".format(hello(name), goodbye(name))
print(hello_goodbye("Alice"))

<b>Hello Alice!</b>


<i>Goodbye Alice.</i>


<div>
<b>Hello Alice!</b>
<i>Goodbye Alice.</i>
</div>


## Timeout(): a real world example

### Tag your functions

In [74]:
def tag(*tags):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        wrapper.tags = tags
        return wrapper
    return decorator

@tag("test", "this is a tag")
def foo():
    pass
print(foo.tags)

('test', 'this is a tag')


### Check the return type

In [80]:
def returns_dict(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        assert(type(result) == dict)
        return result
    return wrapper

@returns_dict
def foo(value):
    return value

try:
    print(foo([1, 2, 3]))
except AssertionError:
    print("foo() did not return a dict!!")
    
def returns(return_type):
    def decorator(func):
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            assert(type(result) == return_type)
        return wrapper
    return decorator

@returns(dict)
def foo(value):
    return value

try:
    print(foo([1,2,3]))
except AssertionError:
    print("foo() did not return a dict!!")

foo() did not return a dict!!
foo() did not return a dict!!
